# Dynamic Customer **Support RAG chatbot** 

This notebook demonstrates:

- Document ingestion
- Embedding generation
- FAISS vector indexing 
- Content retrieval 
- LLM-Based answer generation
- Incremental knowledge update

## ***Imports***

In [ ]:
import os 
from pathlib import Path
from dotenv import load_dotenv

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq

# Loading Environment **Variables**

In [5]:
load_dotenv()

llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.1-8b-instant",
    temperature=0.1
)

## Load **Dataset**

In [12]:
data_path = Path("dataset/et.csv")

loader = CSVLoader(file_path=str(data_path))
documents = loader.load()

print(f"Loaded {len(documents)} Documents.")

Loaded 78 Documents.


## **Embeddings**

In [14]:
embeddings = HuggingFaceBgeEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)

print(" embeddings Model Loaded Sucessfully ")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 852.93it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


 embeddings Model Loaded Sucessfully 


## Created **FAISS** Index

In [16]:
vectordb = FAISS.from_documents(documents,embeddings)

print("FAISS Index Created Sucessfully")

FAISS Index Created Sucessfully


## Created **Retriever** 

In [19]:
retriever = vectordb.as_retriever(search_kwargs = {"k": 3})

## Created **Prompt** Template

In [22]:
prompt = PromptTemplate(
    template="""
    Answer the question ONLY using the context below.
    If the answer is not found, say "I don't know."
    
    CONTEXT
    {context}

    QUESTION
    {question}
    """,

    input_variables=["context","question"]
)

## Build RAG **Chain**

In [ ]:
chain = (
    {
        "context" : retriever,
        "question" : RunnablePassthrough()
    }
)